In [1]:
#Imort delle librerie necessarie
import pandas as pd #per la manipolazione dei dati con DataFrame
import numpy as np #per operazioni numeriche avanzate

from sklearn.ensemble import RandomForestClassifier #per il modello di classificazione

from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures #per il preprocessing dei dati (scaling, encoding, feature engineering)
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV #per la valutazione del modello e la ricerca degli iperparametri

from sklearn.pipeline import Pipeline #per creare pipeline di preprocessing e modellazione
from sklearn.compose import ColumnTransformer #per applicare trasformazioni diverse a colonne diverse

from sklearn.metrics import accuracy_score, make_scorer #per la valutazione delle prestazioni del modello

from imblearn.over_sampling import SMOTE, RandomOverSampler #per il bilanciamento del dataset

from imblearn.under_sampling import RandomUnderSampler #per il bilanciamento del dataset

In [6]:
# step 1 - caricamento del dataset
data = pd.read_csv('../data/Loan_default.csv')

# display delle prime righe del dataset
data.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [7]:
# step 2 - pulizia dei dati
# rimozione delle colonne non necessarie
data = data.drop(columns='LoanID')

# verifico la forma del dataset dopo la rimozione della colonna
print("Shape of dataset after dropping 'LoanID':", data.shape)

Shape of dataset after dropping 'LoanID': (255347, 17)


In [8]:
# step 4 - data processing
# seleziono le feature numeriche per un'analisi futura e l'addestramento del modello
numerical_feature = data.select_dtypes(include=['int','float'])

# display delle feature numeriche
print("Numerical Features:", numerical_feature.columns.tolist())

Numerical Features: ['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio', 'Default']


In [10]:
# step 5 - feature engeneering
# Encoding delle variabili categoriche --> ossia trasformazione in variabili numeriche
# definisco la funzione che mi permette di eseguire l'encoding tramite una mappatura
def ColumnTrans(cat):
    """
        Funzione che mi permette di eseguire l'encoding delle variabili categoriche tramite una mappatura di valori unici a numeri interi.
        Parametri:
            cat: Dataframe
                A Pandas DataFrame contenente le variabili categoriche da trasformare.
        Ritorna:
            cat: Dataframe
                Un Pandas DataFrame con le variabili categoriche trasformate in numeri interi.
    """

    # itero su ogni colonna del DataFrame
    for col in cat.columns:
        # estraggo i valori unici per la colonna
        unique_values = cat[col].unique()

        # creo una mappatura di valori unici a numeri interi
        value_mapping = {value: idx for idx, value in enumerate(unique_values)} # la variabile value_mapping e' quindi un dizionario che associa ad ogni valore unico un numero intero
        # lo fa iterando sui valori unici e assegnando un indice crescente a ciascuno di essi

        # mappo i valori della colonna utilizzando la mappatura creata
        cat[col] = cat[col].map(value_mapping)
    return cat

In [11]:
# step 6 - Data Processing - Encoding delle variabili categoriche
# seleziono le feature categoriche
categorical_feature = data.select_dtypes(include=['object', 'category'])
# eseguo l'encoding delle variabili categoriche
categorical_feature = ColumnTrans(categorical_feature)

# display delle feature categoriche dopo l'encoding
categorical_feature.head()

,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner
0,0,0,0,0,0,0,0
1,1,0,1,1,1,0,0
2,1,1,0,0,0,1,1
3,2,0,1,1,1,2,1
4,0,1,0,1,0,1,1


In [12]:
# step 7 - combino le feature numeriche e categoriche in un unico DataFrame
processed_data = pd.concat([numerical_feature, categorical_feature], axis=1)

# display delle prime righe del DataFrame processato
processed_data.head()

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Default,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner
0,56,85994,50587,520,80,4,15.23,36,0.44,0,0,0,0,0,0,0,0
1,69,50432,124440,458,15,1,4.81,60,0.68,0,1,0,1,1,1,0,0
2,46,84208,129188,451,26,3,21.17,24,0.31,1,1,1,0,0,0,1,1
3,32,31713,44799,743,0,3,7.07,24,0.23,0,2,0,1,1,1,2,1
4,60,20437,9139,633,8,4,6.51,48,0.73,0,0,1,0,1,0,1,1


In [13]:
# step 8 - separazione delle feature e del target
X = processed_data.drop(columns='Default')
y = processed_data['Default']

# display delle forme di X e y
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (255347, 16)
Shape of y: (255347,)


In [ ]:
# step 9 - Inizializzazione delle resempling techniques
# inizializzo le tecniche di reseampling
ros = RandomOverSampler() # Random Over Sampling bilancia il dataset aumentando (duplicando casualmente) le istanze della classe minoritaria
rus = RandomUnderSampler() # Random Under Sampling bilancia il dataset riducendo (eliminando casualmente) le istanze della classe maggioritaria
smote = SMOTE() # Synthetic Minority Over-sampling Technique crea nuove istanze sintetiche della classe minoritaria basate sulle istanze esistenti

In [15]:
# Bilancio il dataset con le diverse tecniche di resempling
x_ros, y_ros = ros.fit_resample(X, y)
print("Shape after Random Over Sampling:", x_ros.shape, y_ros.shape)
x_smote, y_smote = smote.fit_resample(x_ros, y_ros)
print("Shape after SMOTE:", x_smote.shape, y_smote.shape)
x_rus, y_rus = rus.fit_resample(x_smote, y_smote)
print("Shape after Random Under Sampling:", x_rus.shape, y_rus.shape)
# applico in sequenza le tre tecniche di resempling per bilanciare il dataset per affrontare il problema dello sbilanciamento delle classi:
# 1. Over sampling della classe minoritaria
# 2. Creazione di nuove istanze sintetiche della classe minoritaria
# 3. Under sampling della classe maggioritaria


Shape after Random Over Sampling: (451388, 16) (451388,)
Shape after SMOTE: (451388, 16) (451388,)
Shape after Random Under Sampling: (451388, 16) (451388,)


In [16]:
# step 11 - suddivisione del dataset in training e test set
x_train, x_test, y_train, y_test = train_test_split(x_rus, y_rus, test_size=0.2, random_state=42, stratify=y_rus)

In [ ]:
# step 12 - Inizializzazione del modello di classificazione
model = RandomForestClassifier(n_estimators=2000, random_state=42, n_jobs=-1)
print("Model initialized:", model)

Model initialized: RandomForestClassifier(n_estimators=2000, random_state=42)


In [18]:
# step 13 - addestramento del modello
model.fit(x_train, y_train)
print("Model training completed.")

Model training completed.


In [19]:
# step 14 - Previsione sui dati di test
y_pred = model.predict(x_test)
print("Predictions on test data completed.")

Predictions on test data completed.


In [20]:
# dato che il dataset e' sbilanciato controllo se il modello ha predetto entrambe le classi o ha bisogno di piu' lavoro
(y_pred == 0).sum(), (y_pred == 1).sum()

(np.int64(44380), np.int64(45898))

In [23]:
# step 15 evaluazione delle prestazioni del modello
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9904
Precision: 0.9823
Recall: 0.9988
F1 Score: 0.9905
